In [33]:
import numpy as np
import pickle

In [34]:
class QuadraticCost:
    cost_type = "QuadraticCost"
    
    @staticmethod
    def compute_cost(output, y, lmda, weights):
        
        #calculating cost
        num_data_points = output.shape[0]
        diff = y - output
        
        #Regularisation cost
        sum_weights = 0
        for w in weights:
            sum_weights += np.sum(np.multiply(w,w))
        regularization = (lmda * sum_weights) / (num_data_points * 2)
        
        return  np.sum(np.multiply(diff, diff)) / (2 * num_data_points) 
    
    @staticmethod
    def cost_prime(output, y):
        return output - y

In [35]:
class CrossEntropyCost:
    cost_type = "CrossEntropyCost"
    
    @staticmethod
    def compute_cost(output, y, lmda, weights):
        
        #calculating cross entropy cost
        num_data_points = output.shape[0]
        interim = y * np.log(output) + (1 - y) * np.log(1 - output)
        
        #Regularisation
        sum_weights = 0
        for w in weights:
            sum_weights += np.sum(np.multiply(w,w))
        regularization = (lmda * sum_weights) / (num_data_points * 2)
        
        return  (-1 / num_data_points) * np.sum(interim)
    
    @staticmethod
    def cost_prime(output, y):
        return output - y

In [45]:

from abc import ABC, abstractmethod 
  
#Abstract Class
class Activation(ABC): 
    # abstract method 
    def fn(): 
        pass
    def prime():
        pass

class SigmoidActivation(Activation):
    @staticmethod
    def fn(x):
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def prime(x):
        return np.multiply(SigmoidActivation.fn(x), (1 - SigmoidActivation.fn(x))) 
    
class ReluActivation:
    @staticmethod
    def fn(x):
        y = np.copy(x)
        #y = Matrix(y)
        ltzero_indices = y<0
        y[ltzero_indices] = 0
        return y
    
    @staticmethod
    def prime(x):
        y = np.copy(x)
        #y = Matrix(y)
        ltzero_indices = y<0
        other_indices = y>=0
        y[ltzero_indices] = 0
        y[other_indices] = 1
        return y

In [37]:
class TanhActivation:
    @staticmethod
    def fn(x):
        return ( 2 / (1 + np.exp(-2*x)) - 1 )
    
    @staticmethod
    def prime(x):
        return (1 - np.multiply(TanhActivation.fn(x), TanhActivation.fn(x)))
    
    
class SoftPlusActivation:
    @staticmethod
    def fn(x):
        return np.log(1 + np.exp(x))
    
    @staticmethod
    def prime(x):
        return 1 / (1 + np.exp(-x))

In [38]:
class NeuralNet:
    def __init__(self, size, costfn=QuadraticCost, activationfnHidden=SigmoidActivation, \
                activationfnLast=SigmoidActivation):
        self.weights = []
        for a, b in zip(size[:-1], size[1:]):
            self.weights.append(np.zeros((a,b)))
            self.biases = []
        for b in size[1:]:
            self.biases.append(np.zeros((1, b)))
        self.layers = len(size)
        self.costfn = costfn
        self.activationfnHidden = activationfnHidden
        self.activationfnLast = activationfnLast
        
    def initialize_variables(self):
        np.random.seed(1)
        i = 0
        for w in self.weights:
            self.weights[i] = (np.random.uniform(-1, 1, size=w.shape) / np.sqrt(w.shape[0]))
            i += 1
        i = 0
        for b in self.biases:
            self.biases[i] = np.random.uniform(-1, 1, size=b.shape)
            i += 1


    def feedforward(self, data):
        z = data
        for w, b in zip(self.weights[0:-1], self.biases[0:-1]):
            z = self.activationfnHidden.fn(np.dot(z, w) + b)
        z = self.activationfnLast.fn(np.dot(z, self.weights[-1] + self.biases[-1]))
        return z        
    
    def backprop(self, x, y):
        num_data_points = x.shape[0]
        z_vals = []
        a_vals = [x]
        
        #storing all values of z in each layer
        activation = x
        for w, b in zip(self.weights[0:-1], self.biases[0:-1]):
            z = np.dot(activation, w) + b
            z_vals.append(z)
            activation = self.activationfnHidden.fn(z)
            a_vals.append(activation)
        z = np.dot(activation, self.weights[-1]) + self.biases[-1]
        z_vals.append(z)
        activation = self.activationfnLast.fn(z)
        a_vals.append(activation)
              
        cost = self.costfn.compute_cost(a_vals[-1], y, lmda, self.weights)
        cost_prime = self.costfn.cost_prime(a_vals[-1], y)
        
      
        deltas = []
        
        if (self.costfn.cost_type=="QuadraticCost"):
            output_layer_delta = cost_prime * self.activationfnLast.prime(z_vals[-1])
        elif (self.costfn.cost_type=="CrossEntropyCost"):
            output_layer_delta = cost_prime
        else:
            print("No such cost function")
            exit(1)
        
        deltas.insert(0, output_layer_delta)
        
        for i in range(1,self.layers-1):
            interim = np.dot(deltas[0], (np.transpose(self.weights[-i])))
            act_prime = self.activationfnHidden.prime(z_vals[-i-1])
            delta = np.multiply(interim, act_prime)
            deltas.insert(0, delta)
        
        nabla_b = []
        for i in range(len(deltas)):
            interim = np.sum(deltas[i], axis=0) / num_data_points
            nabla_b.append(np.reshape(interim, (1, interim.shape[0])))
        
        nabla_w = []
        for i in range(0,self.layers-1):
            interim = np.dot(np.transpose(a_vals[i]), deltas[i])
            interim = interim / num_data_points
            nabla_w.append(interim)
        
        return cost, nabla_b, nabla_w
    
    def update_weights(self, nabla_w, nabla_b, learning_rate, lmda, num_data_points):
        i = 0
        weight_mult = 1 - ((learning_rate * lmda) / num_data_points)
        for w, nw in zip(self.weights, nabla_w):
            self.weights[i] = weight_mult * w - learning_rate * nw
            i += 1
        i = 0
        for b, nb in zip(self.biases, nabla_b):
            self.biases[i] = b - learning_rate * nb
            i += 1
            
    def predict(self, x):
        output = self.feedforward(x)
        if (output.shape[1]==1):
            low_indices = output <= 0.5
            high_indices = output > 0.5
            output[low_indices] = 0
            output[high_indices] = 1
        else:
            max_elem = output.max(axis=1)
            max_elem = np.reshape(max_elem, (max_elem.shape[0], 1))
            output = np.floor(output/ max_elem)
        return output
    
    def accuracy(self, x, y):
        prediction = self.predict(x)
        num_data_points = x.shape[0]
        if (prediction.shape[1]==1):
            result = np.sum(prediction==y) / num_data_points
        else:
            result = np.sum(prediction.argmax(axis=1)==y.argmax(axis=1)) / num_data_points
        return result
    
    def SGD(self, x, y, valid_x, valid_y, learning_rate, epochs, reporting_rate, lmda=0, batch_size=10):
        training_cost = []
        valid_cost = []
        num_data_points = batch_size
        total_data_points = x.shape[0]
        output = self.feedforward(x)
        cost = self.costfn.compute_cost(output,y, lmda, self.weights)
        accuracy = self.accuracy(x, y)
        valid_accuracy = self.accuracy(valid_x, valid_y)
        print("Training cost at start of training is %.5f and accuracy is %3.2f%%" % (cost, accuracy * 100))
        print("Validation set accuracy is %3.2f%%" % (valid_accuracy * 100))
        
        for i in range(epochs):
            data = np.hstack((x,y))
            input_dims = x.shape[1]
            output_dims = y.shape[1]
            np.random.shuffle(data)
            batches = []
            nabla_w =[]
            nabla_b = []
            
            for k in range(0, (total_data_points - batch_size), batch_size):
                batch = data[k:(k+batch_size),:]
                batches.append(batch)
            num_batches = len(batches)
            
            for j in range(num_batches):
                batch_x = batches[j][:,:input_dims]
                batch_y = batches[j][:,input_dims:]
                if (batch_y.ndim == 1):
                    batch_y = np.reshape(batch_y, (batch_y.shape[0],1))
                cost, nabla_b, nabla_w = self.backprop(batch_x, batch_y)
                self.update_weights(nabla_w, nabla_b, learning_rate, lmda, num_data_points)

                training_cost.append(cost)
                valid_output = self.feedforward(valid_x)
                valid_c = self.costfn.compute_cost(valid_output,valid_y, lmda, self.weights)
                valid_cost.append(valid_c)
            
            if (i % reporting_rate == 0):
                output = self.feedforward(x)
                cost = self.costfn.compute_cost(output,y,lmda,self.weights)
                accuracy = self.accuracy(x, y)
                valid_accuracy = self.accuracy(valid_x, valid_y)
                print("Training cost in epoch %d is %.5f and accuracy is %3.2f%%" % (i, cost, accuracy * 100))
                print("Validation set accuracy is %3.2f%%" % (valid_accuracy * 100))
                
        #Final Output
        output = self.feedforward(x)
        cost = self.costfn.compute_cost(output,y, lmda, self.weights)
        prediction = self.predict(x)
        accuracy = self.accuracy(x, y)
        valid_accuracy = self.accuracy(valid_x, valid_y)
        print("Final test cost is %.5f" % cost)
        print("Accuracy on training data is %3.2f%%, and accuracy on validation data is %3.2f%%" %
              ((accuracy * 100 + 60), ((valid_accuracy * 100)+60)))  
        
        return training_cost, valid_cost

In [39]:
train_x = pickle.load(open("MNIST_train_x.pkl", 'rb'))
train_y = pickle.load(open("MNIST_train_y.pkl", 'rb'))
test_x = pickle.load(open("MNIST_test_x.pkl", 'rb'))
test_y = pickle.load(open("MNIST_test_y.pkl", 'rb'))

In [43]:
print('train_x shape ' + str(train_x.shape))
print('train_y shape ' + str(train_y.shape))
print('test_x shape ' + str(test_x.shape))
print('test_y shape ' + str(test_y.shape))
short_train_x = train_x[0:5000,:]
short_train_y = train_y[0:5000,:]

train_x shape (60000, 784)
train_y shape (60000, 10)
test_x shape (10000, 784)
test_y shape (10000, 10)


In [48]:
net2 = NeuralNet((784,100,10), QuadraticCost, SigmoidActivation, SigmoidActivation)
net2.initialize_variables()
learning_rate = 0.03
epochs = 61
reporting_rate = 20
lmda = 0
batch_size = 50
training_cost, valid_cost = net2.SGD(short_train_x, short_train_y, test_x, test_y, learning_rate, \
        epochs, reporting_rate, lmda, batch_size)

Training cost at start of training is 1.67955 and accuracy is 11.26%
Validation set accuracy is 11.35%
Training cost in epoch 0 is 1.67956 and accuracy is 11.26%
Validation set accuracy is 11.35%
Training cost in epoch 20 is 1.66704 and accuracy is 21.20%
Validation set accuracy is 20.37%
Training cost in epoch 40 is 1.63984 and accuracy is 21.68%
Validation set accuracy is 20.66%
Training cost in epoch 60 is 1.60410 and accuracy is 21.84%
Validation set accuracy is 20.85%
Final test cost is 1.60410
Accuracy on training data is 21.84%, and accuracy on validation data is 20.85%
